# Convolution Neural Network

Note that this is an add-on to MLUdemy/8_DeepLearning 

### Part 1 - Building the CNN 

Sequential - initialize our nn as sequence of layers, cnn is sequence of layers 
- 1st step to making cnn, add convolutional layers, since we're working on images (2d objects), videos are 3d
- 2nd step - add pooling layers in 2d
- 3rd step - flatten to convert pool feature maps thru convo and maxpooling to become feature vector
- 4th step - add connected layers and ann

In [6]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense

In [2]:
# initialize CNN
# making a clf some images if dog or cat
classifier = Sequential()
# cnn is initialized

In [3]:
# step 1 - convolution
# 32,3,3 -> 32 feature detectors, 3 x 3 dimensions 
# input_shape = expected format of our images (colored image 3d, black and white image 2d) 
# (64,64,3) -> 64x64 size image, 3 = color image, 2 = black and white -> number of channels, 
# be careful tensorflow backend and theano backend are different format for above line is different
# use rectify/relu activation function because we need nonlinearity for images 
classifier.add(Convolution2D(32,(3,3), input_shape = (64,64,3), activation = 'relu'))

In [7]:
# step 2 - pooling 
# pooling is used to reduce size of feature map by 2 (divided by 2)
classifier.add(MaxPooling2D(pool_size=(2,2)))

In [8]:
# Add another convolution layer
# at this point we have the pooled feature maps, so we don't need input_shape
# then you can change feature detectors to a different value for increased accuracy 
classifier.add(Convolution2D(32,(3,3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))

In [9]:
# step 3 - flatten
# feature map flattening into one vector 
# high numbers represent spatial structure (or specific detail) of input image  
# feature map corresponds to one specific feature of the image, each node that has a high number
classifier.add(Flatten())

In [10]:
# Step 4 - full connection
# number of nodes in hidden layer choose between # of input nodes and # of output nodes
# activate neurons with this activation function 
classifier.add(Dense(units = 128, activation = 'relu'))
# cat or dog is binary outcome (soft max for more than two outcomes)
# expect one node that tells you dog or cat 
classifier.add(Dense(units = 1, activation = 'sigmoid'))


In [11]:
# Compiling the CNN 
# adam algo, loss func is binary crossentropy, because classification problem and binary outcome (categorical_crossentropy for more than two)
# performance metric - accuracy is most common one
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Part 2 - Fitting the CNN to the images # Part  

In [12]:
# image augmentation allows us to enrich our datasets without adding more images with little or no overfitting
from keras.preprocessing.image import ImageDataGenerator

# image scaling, shearing = geometric transformation, zoom range = random zoom, horizontal_flip = flip images 
train_datagen = ImageDataGenerator(
        #rescale pixel values between 0 or 1, because pixels take value from 0 to 255
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
# rescale images of our test set
test_datagen = ImageDataGenerator(rescale=1./255)
# train and valid generator create training and test/validation set 
training_set = train_datagen.flow_from_directory(
        'dataset/training_set',
        # dimensions expected from CNN input_shape (64,64,3)
        target_size=(64, 64),
        # number of images that will go through CNN
        batch_size=32,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

# fit cnn onto training set and testing peformance on test set
# steps_per_epoch = no of train samples/batch_size and validation_steps = no of validation samples/batch_size
classifier.fit_generator(
        training_set,
        steps_per_epoch=8000/32,
        epochs=25,
        validation_data=test_set,
        validation_steps=2000/32)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/25
250/250 [==============================] - 57s 227ms/step - loss: 0.6886 - acc: 0.5480 - val_loss: 0.6714 - val_acc: 0.5640
Epoch 2/25
250/250 [==============================] - 54s 215ms/step - loss: 0.6293 - acc: 0.6475 - val_loss: 0.6034 - val_acc: 0.6734
Epoch 3/25
250/250 [==============================] - 54s 217ms/step - loss: 0.5833 - acc: 0.6933 - val_loss: 0.5653 - val_acc: 0.7082
Epoch 4/25
250/250 [==============================] - 54s 215ms/step - loss: 0.5462 - acc: 0.7229 - val_loss: 0.5515 - val_acc: 0.7303
Epoch 5/25
250/250 [==============================] - 53s 211ms/step - loss: 0.5280 - acc: 0.7339 - val_loss: 0.5424 - val_acc: 0.7243
Epoch 6/25
250/250 [==============================] - 53s 210ms/step - loss: 0.5068 - acc: 0.7489 - val_loss: 0.5053 - val_acc: 0.7540
Epoch 7/25
250/250 [==============================] - 52s 209ms/step - loss: 0.4912 - acc: 0.7647 - val_lo

Next steps
- add another convolution layer or/and
- add another fully connected layer to get 80% accuracy 
- insert below after first pooling

### Part 3 - Making new predictions

In [13]:
import numpy as np
from keras.preprocessing import image

In [14]:
# must match target size
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg',target_size = (64,64))

In [15]:
# image of 2-d (64,64) to 3-d array (64,64,3) 
# matches exactly as the input 
test_image = image.img_to_array(test_image)

In [17]:
# ValueError: Error when checking input: expected conv2d_1_input to have 4 dimensions, but got array with shape (64, 64, 3)
# therefore need to add dimension
# this missing dimension corresponds to the batch that will contain how many inputs (1 or many)
test_image = np.expand_dims(test_image, axis = 0)
# now we have a (1,64,64,3)

In [19]:
result = classifier.predict(test_image)

In [21]:
training_set.class_indices

{'cats': 0, 'dogs': 1}

In [23]:
if result[0][0] == 1:
    prediction = 'dog'
else: 
    prediction = 'cat'
prediction

'dog'

In [29]:
# must match target size
test_image = image.load_img('dataset/single_prediction/cat_or_dog_2.jpg',target_size = (64,64))

In [30]:
# image of 2-d (64,64) to 3-d array (64,64,3) 
# matches exactly as the input 
test_image = image.img_to_array(test_image)

In [31]:
# ValueError: Error when checking input: expected conv2d_1_input to have 4 dimensions, but got array with shape (64, 64, 3)
# therefore need to add dimension
# this missing dimension corresponds to the batch that will contain how many inputs (1 or many)
test_image = np.expand_dims(test_image, axis = 0)
# now we have a (1,64,64,3)

In [34]:
result = classifier.predict(test_image)
result

array([[ 0.]], dtype=float32)

In [36]:
if int(result[0][0]) == 1:
    prediction = 'dog'
else: 
    prediction = 'cat'
prediction

'cat'